In [1]:
import pandas as pd

dados = pd.read_csv("https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv")

renomear = {'mileage_per_year' : 'milhas_por_ano',
         'model_year' : 'modelo_do_ano', 
         'price': 'preco',
         'sold':'vendido'}

dados = dados.rename(columns = renomear)
dados.head()

#trocar yes e no por 1 e 0 

troca = {'yes': 1,
         'no': 0}
dados.vendido = dados.vendido.map(troca)
dados.head()

#trabalhar com a idade do modelo 

from datetime import datetime
ano_atual = datetime.today().year

dados['idade_do_modelo'] = ano_atual - dados.modelo_do_ano 
dados ['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano", "modelo_do_ano"], axis=1)


dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,23,35085.22134
1,40557.96,1,25,12622.05362
2,89627.50,0,17,11440.79806
3,95276.14,0,8,43167.32682
4,117384.68,1,9,12770.11290


In [2]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

# Validação cruzada



In [3]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=5) #cv é quantos pedaçs eu quero quebrar a validação cruzada
#uma escolha entre 5 ou 10 de cv já é suficiente: como dizem artigos ciemtificos
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation 3 : [%.2f , %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy com cross validation 3 : [75.21 , 76.35]


# Aletoriedade no cross validate

In [4]:
def imprime_resul(results):
  media = results ['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("media %.2f" % (media*100) )
  print("Accuracy com cross validation : [%.2f , %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))


In [5]:
from sklearn.model_selection import KFold

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv) #cv é quantos pedaçs eu quero quebrar a validação cruzada
#uma escolha entre 5 ou 10 de cv já é suficiente: como dizem artigos ciemtificos

imprime_resul(results)

media 75.78
Accuracy com cross validation : [74.37 , 77.19]


In [6]:
#para ter aletoriedade 

import numpy as np

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv) #cv é quantos pedaçs eu quero quebrar a validação cruzada
#uma escolha entre 5 ou 10 de cv já é suficiente: como dizem artigos ciemtificos

imprime_resul(results)

media 75.76
Accuracy com cross validation : [73.26 , 78.26]


In [7]:
#para ter aletoriedade 
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv) #cv é quantos pedaçs eu quero quebrar a validação cruzada
#uma escolha entre 5 ou 10 de cv já é suficiente: como dizem artigos ciemtificos

imprime_resul(results)

media 75.76
Accuracy com cross validation : [73.26 , 78.26]


In [8]:
#para ter aletoriedade 
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv) #cv é quantos pedaçs eu quero quebrar a validação cruzada
#uma escolha entre 5 ou 10 de cv já é suficiente: como dizem artigos ciemtificos

imprime_resul(results)

media 75.76
Accuracy com cross validation : [73.26 , 78.26]


In [9]:
 #gerar modelo de carros de acordo com a idade do modelos

np.arange(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2,2,10000)
dados.modelo_aleatorio.unique()

array([22, 23, 16,  9,  7, 15, 20, 14, 24, 21, 13, 17, 18,  8, 12, 19, 11,
       25, 26, 10,  6,  5,  4])

# Validação cruzada com GroupKfold


In [10]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
4999,74023.29,0,17,24812.80412,18
5322,84843.49,0,18,23095.63834,18
5319,83100.27,0,24,36240.72746,25
5316,87932.13,0,21,32249.56426,19
5315,77937.01,0,20,28414.50704,19


In [11]:
#usar validador que separe teste e treino em grupos

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv,groups =dados.modelo_aleatorio , return_train_score=False) #temos que especificar quais grupos usares para validação
imprime_resul(results)

media 75.77
Accuracy com cross validation : [72.97 , 78.58]


#Cross validation com StandardScaler


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

# scaler = StandardScaler()
# scaler.fit(raw_treino_x)
# treino_x = scaler.transform(raw_treino_x)
# teste_x = scaler.transform(raw_teste_x)

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [16]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi de %.2f %%" % acuracia)

A acurácia foi de 77.48 %


In [17]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler =StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)
result = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo_aleatorio, return_train_score=False)
imprime_resul(results)

media 75.77
Accuracy com cross validation : [72.97 , 78.58]
